In [24]:
# Written by Dr Daniel Buscombe, Marda Science LLC
import os
# from prediction_imports import *
from doodleverse_utils.prediction_imports import *
#from imports import *
from doodleverse_utils.imports import *
import pandas as pd
#=======================================================

USE_GPU = False

if USE_GPU == True:
   ##use the first available GPU
   os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
else:
   ## to use the CPU (not recommended):
   os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

weights='C:/1_USGS/CoastSeg/repos/6_sniffer-classifer/Sniffer-Classifier/Dan_model/weights/resunet/ct_ortho_all_water_768.h5'


In [2]:
W=[]
W.append(weights)

M= []; C=[]; T = []
for counter,weights in enumerate(W):
    print(f"{counter}:{weights} ")
    try:
        configfile = weights.replace('_fullmodel.h5','.json').replace('weights', 'config')
        with open(configfile) as f:
            config = json.load(f)
            # print(config)
    except:
        configfile = weights.replace('.h5','.json').replace('weights', 'config')
        with open(configfile) as f:
            config = json.load(f)
            # print(config)

    for k in config.keys():
        expression=k+'=config["'+k+'"]'
        # print(expression)
        # print(k)
        exec(k+'=config["'+k+'"]')
        # This exec dynamically creates all variables out of all the keys in config
    print('.....................................')
    print('Creating and compiling model {}...'.format(counter))
    if MODEL =='resunet':
        model =  custom_resunet((TARGET_SIZE[0], TARGET_SIZE[1], N_DATA_BANDS),
                        FILTERS,
                        nclasses=[NCLASSES+1 if NCLASSES==1 else NCLASSES][0],
                        kernel_size=(KERNEL,KERNEL),
                        strides=STRIDE,
                        dropout=DROPOUT,#0.1,
                        dropout_change_per_layer=DROPOUT_CHANGE_PER_LAYER,#0.0,
                        dropout_type=DROPOUT_TYPE,#"standard",
                        use_dropout_on_upsampling=USE_DROPOUT_ON_UPSAMPLING,#False,
                        )
    try:
        model = tf.keras.models.load_model(weights)
    except:
        # model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [mean_iou, dice_coef])
        model.compile(optimizer = 'adam', loss = dice_coef_loss, metrics = [mean_iou, dice_coef])
        model.load_weights(weights)

        M.append(model)
        C.append(configfile)
        T.append(MODEL)

metadatadict = {}
metadatadict['model_weights'] = W
metadatadict['config_files'] = C
metadatadict['model_types'] = T

0:C:/1_USGS/CoastSeg/repos/6_sniffer-classifer/Sniffer-Classifier/Dan_model/weights/resunet/ct_ortho_all_water_768.h5 
.....................................
Creating and compiling model 0...


In [3]:
with open(configfile) as f:
    config = json.load(f)
config

{'TARGET_SIZE': [768, 768],
 'MODEL': 'resunet',
 'NCLASSES': 1,
 'KERNEL': 9,
 'STRIDE': 2,
 'BATCH_SIZE': 8,
 'FILTERS': 6,
 'N_DATA_BANDS': 3,
 'DROPOUT': 0.1,
 'DROPOUT_CHANGE_PER_LAYER': 0.0,
 'DROPOUT_TYPE': 'standard',
 'USE_DROPOUT_ON_UPSAMPLING': False,
 'DO_TRAIN': True,
 'LOSS': 'cat',
 'PATIENCE': 10,
 'MAX_EPOCHS': 100,
 'VALIDATION_SPLIT': 0.7,
 'RAMPUP_EPOCHS': 20,
 'SUSTAIN_EPOCHS': 0.0,
 'EXP_DECAY': 0.9,
 'START_LR': 1e-07,
 'MIN_LR': 1e-07,
 'MAX_LR': 0.0001,
 'FILTER_VALUE': 0,
 'DOPLOT': True,
 'ROOT_STRING': 'ct_ortho_all_water_768',
 'USEMASK': False,
 'AUG_ROT': 5,
 'AUG_ZOOM': 0.05,
 'AUG_WIDTHSHIFT': 0.05,
 'AUG_HEIGHTSHIFT': 0.05,
 'AUG_HFLIP': True,
 'AUG_VFLIP': False,
 'AUG_LOOPS': 10,
 'AUG_COPIES': 5,
 'REMAP_CLASSES': {'0': 0,
  '1': 0,
  '2': 1,
  '3': 1,
  '4': 1,
  '5': 1,
  '6': 1,
  '7': 1,
  '8': 1,
  '9': 1,
  '10': 1,
  '11': 1,
  '12': 1}}

In [4]:
base_model=model
base_model

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# keras.utils.plot_model(base_model, "my_first_model_with_shape_info.png", show_shapes=True)

In [6]:
base_model.layers[-1].activation
base_model.outputs
base_model.layers[-1]

In [7]:
last_layer=base_model.get_layer('conv2d_29')
last_layer

In [8]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx

In [9]:
base_model.get_layer("add_4").get_config()

{'name': 'add_4', 'trainable': True, 'dtype': 'float32'}

In [38]:
base_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [10]:
idx=getLayerIndexByName(base_model,"add_4")
idx

56

In [11]:
base_model.layers[idx].output

<KerasTensor: shape=(None, 48, 48, 96) dtype=float32 (created by layer 'add_4')>

In [12]:
from keras.models import Model
print(base_model.layers[idx].get_config())
print(base_model.layers[idx].output)
cut_base_model=Model(inputs=base_model.inputs,outputs=base_model.layers[idx].output)

{'name': 'add_4', 'trainable': True, 'dtype': 'float32'}
KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 96), dtype=tf.float32, name=None), name='add_4/add:0', description="created by layer 'add_4'")


In [13]:
cut_base_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [14]:
# Keep the base_model for transfer learning
cut_base_model.trainable=False

In [15]:
cut_base_model.compile(optimizer = 'adam', loss = dice_coef_loss, metrics = [mean_iou, dice_coef])

In [16]:
# Validate that the trainable attribute of the batch_normalization layer is set to false
cut_base_model.get_layer("batch_normalization_12").trainable

False

In [17]:
img_shape = (768, 768)
inputs=keras.Input(img_shape+(3,))
inputs

<KerasTensor: shape=(None, 768, 768, 3) dtype=float32 (created by layer 'input_2')>

In [37]:
cut_base_model.layers[-1].output

<KerasTensor: shape=(None, 48, 48, 96) dtype=float32 (created by layer 'add_4')>

In [36]:
cut_base_model.input

<KerasTensor: shape=(None, 768, 768, 3) dtype=float32 (created by layer 'input_1')>

In [60]:
cut_base_model.output

<KerasTensor: shape=(None, 48, 48, 96) dtype=float32 (created by layer 'add_4')>

In [42]:
x=cut_base_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.Dense(1,activation="sigmoid")(x)

new_model = Model(inputs=cut_base_model.input,outputs=x)

In [43]:
new_model.summary(show_trainable=True)

Model: "model_4"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [44]:
new_model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy()])

In [45]:
new_model.summary(show_trainable=True)

Model: "model_4"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [46]:
type(new_model)

keras.engine.functional.Functional

In [47]:
def save_model_data(model_name:str,model:'keras.engine.functional.Functional'):
    # Create a directory to store all the model data
    model_save_path=os.getcwd()+os.sep+"models"
    if not os.path.exists(model_save_path):
        os.mkdir(model_save_path)
    model_save_path=os.getcwd()+os.sep+"models"+os.sep+model_name
    if os.path.exists(model_save_path):
        raise FileExistsError(f"This folder '{model_name}' already exists: {model_save_path}.\n Please rename your model.")
    else:
        os.mkdir(model_save_path)
    # Save the actual model
    model.save(model_save_path+os.sep+"model")

In [49]:
# Enter the name of your model
model_name="transfer_model_v_1_1"
# This might take a few seconds
save_model_data(model_name,new_model)

INFO:tensorflow:Assets written to: c:\1_USGS\CoastSeg\repos\6_sniffer-classifer\Sniffer-Classifier\models\transfer_model_v_1_1\model\assets
